<h1 align="center">Introduction to Machine Learning - 25737-2</h1>
<h4 align="center">Dr. R. Amiri</h4>
<h4 align="center">Sharif University of Technology, Spring 2024</h4>


**<font color='red'>Plagiarism is strongly prohibited!</font>**


**Student Name**: Parsa Yousefpoor

**Student ID**: 400104686





## Importing Libraries

First we import libraries that we need for this assignment.

In [16]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
'''from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.constraints import maxnorm
from keras import regularizers'''
# import any other libraries needed below

'from keras.models import Sequential\nfrom keras.layers import Dense, Dropout\nfrom keras.optimizers import RMSprop\nfrom keras.constraints import maxnorm\nfrom keras import regularizers'

## Reading Data and Preprocessing

In this section, we want to read data from a CSV file and then preprocess it to make it ready for the rest of the problem.

First, we read the data in the cell below and extract an $m \times n$ matrix, $X$, and an $m \times 1$ vector, $Y$, from it, which represent our knowledge about the features of the data (`X1`, `X2`, `X3`) and the class (`Y`), respectively. Note that by $m$, we mean the number of data points and by $n$, we mean the number of features.

In [20]:
X, Y = None, None

### START CODE HERE ###
data = pd.read_csv('data_logistic.csv')
X = data[['X1', 'X2', 'X3']]
Y = data['Y']
### END CODE HERE ###

print(X.shape)
print(Y.shape)

(10000, 3)
(10000,)


Next, we should normalize our data. For normalizing a vector $\mathbf{x}$, a very common method is to use this formula:

$$
\mathbf{x}_{norm} = \dfrac{\mathbf{x} - \overline{\mathbf{x}}}{\sigma_\mathbf{x}}
$$

Here, $\overline{x}$ and $\sigma_\mathbf{x}$ denote the mean and standard deviation of vector $\mathbf{x}$, respectively. Use this formula and store the new $X$ and $Y$ vectors in the cell below.

**Question**: Briefly explain why we need to normalize our data before starting the training.

**Answer**: 

**First**: to standardize

**Second**: to eliminate its effect in distances

In [21]:
### START CODE HERE ###
X = (X - np.mean(X)) / np.std(X)
Y = (Y - np.mean(Y)) / np.std(Y)
### END CODE HERE ###

C:\Users\Microsoft\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Finally, we should add a column of $1$s at the beginning of $X$ to represent the bias term. Do this in the next cell. Note that after this process, $X$ should be an $m \times (n+1)$ matrix.

In [22]:
### START CODE HERE ###
m, n = X.shape
X = np.hstack((np.ones((m, 1)), X))
### END CODE HERE ###

print(X.shape)

(10000, 4)


## Training Model

### Sigmoid Function
You should begin by implementing the $\sigma(\mathbf{x})$ function. Recall that the logistic regression hypothesis $\mathcal{h}()$ is defined as:
$$
\mathcal{h}_{\theta}(\mathbf{x}) = \mathcal{g}(\theta^\mathbf{T}\mathbf{x})
$$
where $\mathcal{g}()$ is the sigmoid function as:
$$
\mathcal{g}(\mathbf{z}) = \frac{1}{1+exp^{-\mathbf{z}}}
$$
The Sigmoid function has the property that $\mathbf{g}(+\infty)\approx 1$ and $\mathcal{g}(−\infty)\approx0$. Test your function by calling `sigmoid(z)` on different test samples. Be certain that your sigmoid function works with both vectors and matrices - for either a vector or a matrix, your function should perform the sigmoid function on every element.

In [23]:
def sigmoid(self, Z):
    '''
    Applies the sigmoid function on every element of Z
    Arguments:
        Z can be a (n,) vector or (n , m) matrix
    Returns:
        A vector/matrix, same shape with Z, that has the sigmoid function applied elementwise
    '''
    ### START CODE HERE ###
    A = 1 / (1 + np.exp(-Z))
    return A
    ### END CODE HERE ###

### Cost Function 
Implement the functions to compute the cost function. Recall the cost function for logistic regression is a scalar value given by:
$$
\mathcal{J}(\theta) = \sum_{i=1}^{n}[-y^{(i)}\log{(\mathcal{h}_\theta(\mathbf{x}^{(i)}))}-(1-y^{(i)})\log{(1-\mathcal{h}_\theta(\mathbf{x}^{(i)}))}] + \frac{\lambda}{2}||\theta||_2^2
$$

In [24]:
def computeCost(theta, X, y, regLambda):
    '''
    Computes the objective function
    Arguments:
        theta is d-dimensional numpy vector
        X is a n-by-d numpy matrix
        y is an n-dimensional numpy vector
        regLambda is the scalar regularization constant
    Returns:
        a scalar value of the cost  ** make certain you're not returning a 1 x 1 matrix! **
    '''
    
    m, n = X.shape
    loss = None
    ### START CODE HERE ###
    m = len(y)
    h = 1 / (1 + np.exp(-(X @ theta)))  # Predicted probabilities
    term1 = -y * np.log(h)
    term2 = -(1 - y) * np.log(1 - h)
    regularization_term = (regLambda / (2 * m)) * np.sum(theta[1:]**2)  # Exclude bias term

    # Compute the cost (without regularization)
    cost = (1 / m) * np.sum(term1 + term2)

    # Add regularization term
    cost += regularization_term

    loss = cost
    ### END CODE HERE ###
    return loss

### Gradient of the Cost Function
Now, we want to calculate the gradient of the cost function. The gradient of the cost function is a d-dimensional vector.\
We must be careful not to regularize the $\theta_0$ parameter (corresponding to the first feature we add to each instance), and so the 0's element is given by:
$$
\frac{\partial \mathcal{J}(\theta)}{\partial \theta_0} = \sum_{i=1}^n (\mathcal{h}_\theta(\mathbf{x}^{(i)})-y^{(i)})
$$

Question: What is the answer to this problem for the $j^{th}$ element (for $j=1...d$)?

Answer:

In [25]:
def computeGradient(theta, X, y, regLambda):
    '''
    Computes the gradient of the objective function
    Arguments:
        theta is d-dimensional numpy vector
        X is a n-by-d numpy matrix
        y is an n-dimensional numpy vector
        regLambda is the scalar regularization constant
    Returns:
        the gradient, an d-dimensional vector
    '''
    
    m, n = X.shape
    grad = None
    ### START CODE HERE ###
    m, n = X.shape  # Number of training examples and features
    grad = np.zeros(n)  # Initialize gradient vector
    
    # Compute predictions (hypothesis)
    y_pred = np.dot(X, theta)
    
    # Compute the gradient
    grad = -(1/m) * np.dot(X.T, y - y_pred) + (regLambda/m) * theta
    ### END CODE HERE ###
    return grad

### Training and Prediction
Once you have the cost and gradient functions complete, implement the fit and predict methods.\
Your fit method should train the model via gradient descent, relying on the cost and gradient functions. This function should return two parameters. The first parameter is $\theta$, and the second parameter is a `numpy` array that contains the loss in each iteration. This array is indicated by `loss_history` in the code.\
Instead of simply running gradient descent for a specific number of iterations, we will use a more sophisticated method: we will stop it after the solution hasconverged. Stop the gradient descent procedure when $\theta$ stops changing between consecutive iterations. You can detect this convergence when:
$$
||\theta_{new}-\theta_{old}||_2 <= \epsilon,
$$
for some small $\epsilon$ (e.g, $\epsilon=10E-4$).\
For readability, we’d recommend implementing this convergence test as a dedicated function `hasConverged`.

In [26]:
def fit(X, y, regLambda = 0.01, alpha = 0.01, epsilon = 1e-4, maxNumIters = 100):
    '''
    Trains the model
    Arguments:
        X           is a n-by-d numpy matrix
        y           is an n-dimensional numpy vector
        maxNumIters is the maximum number of gradient descent iterations
        regLambda   is the scalar regularization constant
        epsilon     is the convergence rate
        alpha       is the gradient descent learning rate
    '''
    
    m, n = X.shape
    theta, loss_history = None, None 
    ### START CODE HERE ###
    m, n = X.shape
    theta = np.zeros(n)  # Initialize theta with zeros
    loss_history = []    # To store loss values in each iteration
    
    for _ in range(maxNumIters):
        # Compute the gradient
        gradient = computeGradient(theta, X, y, regLambda)
        
        # Update theta using gradient descent
        theta -= alpha * gradient
        
        # Compute the cost (loss) and append it to loss_history
        cost = computeCost(theta, X, y, regLambda)
        loss_history.append(cost)
        
        # Check for convergence
        if hasConverged(theta, theta - alpha * gradient, epsilon):
            break
    loss_history = np.array(loss_history)
    ### END CODE HERE ###
    return theta, loss_history




def hasConverged(theta_old, theta_new, epsilon):
    '''
    Return if the theta converged or not
    Arguments:
        theta_old   is the theta calculated in prevoius iteration
        theta_new   is the theta calculated in current iteration
        epsilon     is the convergence rate
    '''
    
    ### START CODE HERE ###
    if np.linalg.norm(theta_new - theta_old) <= epsilon:
        return True
    ### END CODE HERE ###
    return False

Finally, we want to evaluate our loss for this problem. Complete the cell below to calculate and print the loss of each iteration and the final theta of your model.

In [27]:
theta, loss_history = fit(X, Y) # calculating theta and loss of each iteration

### START CODE HERE ###
# Print the loss for each iteration
for i, loss in enumerate(loss_history):
    print(f"Iteration {i+1}: Loss = {loss:.6f}")

# Print the final theta
print(f"Final theta: {theta}")
### END CODE HERE ###

Iteration 1: Loss = 0.686920
Iteration 2: Loss = 0.680771
Iteration 3: Loss = 0.674700
Iteration 4: Loss = 0.668705
Iteration 5: Loss = 0.662785
Iteration 6: Loss = 0.656939
Iteration 7: Loss = 0.651166
Iteration 8: Loss = 0.645466
Iteration 9: Loss = 0.639836
Iteration 10: Loss = 0.634276
Iteration 11: Loss = 0.628786
Iteration 12: Loss = 0.623363
Iteration 13: Loss = 0.618008
Iteration 14: Loss = 0.612719
Iteration 15: Loss = 0.607495
Iteration 16: Loss = 0.602335
Iteration 17: Loss = 0.597239
Iteration 18: Loss = 0.592206
Iteration 19: Loss = 0.587234
Iteration 20: Loss = 0.582323
Iteration 21: Loss = 0.577473
Iteration 22: Loss = 0.572681
Iteration 23: Loss = 0.567948
Iteration 24: Loss = 0.563272
Iteration 25: Loss = 0.558653
Iteration 26: Loss = 0.554091
Iteration 27: Loss = 0.549583
Iteration 28: Loss = 0.545130
Iteration 29: Loss = 0.540731
Iteration 30: Loss = 0.536385
Iteration 31: Loss = 0.532091
Iteration 32: Loss = 0.527848
Iteration 33: Loss = 0.523657
Iteration 34: Loss 

### Testing Your Implementation
To test your logistic regression implementation, first you should use `train_test_split` function to split dataset into three parts:

- 70% for the training set
- 20% for the validation set
- 10% for the test set

Do this in the cell below.

In [28]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = None, None, None, None, None, None

### START CODE HERE ###
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.33, random_state=42)
### END CODE HERE ###

Then, you should complete `predict` function to find the weight vector and the loss on the test data.

In [29]:
def predict(X, theta):
    '''
    Use the model to predict values for each instance in X
    Arguments:
        theta is d-dimensional numpy vector
        X     is a n-by-d numpy matrix
    Returns:
        an n-dimensional numpy vector of the predictions, the output should be binary (use h_theta > .5)
    '''
    
    Y = None
    ### START CODE HERE ###
    # Compute the dot product of X and theta
    z = np.dot(X, theta)
    
    # Apply the sigmoid function to get probabilities
    h_theta = 1 / (1 + np.exp(-z))
    
    # Convert probabilities to binary predictions
    Y = (h_theta > 0.5).astype(int)
    ### END CODE HERE ###
    return Y

Now, run the `fit` and `predict` function for different values of the learning rate and regularization constant. Plot the `loss_history` of these different values for train and test data both in the same figure.

**Question**: Discuss the effect of the learning rate and regularization constant and find the best values of these parameters.

**Answer**:

In [30]:
### START CODE HERE ###
def create_model(neurons=379, init_mode='uniform', activation='relu', inputDim=8040,
                 dropout_rate=1.1, learn_rate=0.001, momentum=0.7, weight_constraint=6):
    model = Sequential()
    model.add(Dense(neurons, input_dim=inputDim, kernel_initializer=init_mode,
                    activation=activation, kernel_constraint=maxnorm(weight_constraint),
                    kernel_regularizer=regularizers.l2(0.002)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = RMSprop(lr=learn_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define k-fold cross-validation test harness
seed = 7
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

# Initialize lists to store loss history
train_loss_history = []
val_loss_history = []

Y_train_binary = (Y_train > Y_train.median()).astype(int)
for train, val in kfold.split(X_train, Y_train_binary):
    model = create_model()  # Create a new model for each fold
    history = model.fit(X_train[train], Y_train_binary[train], validation_data=(X_train[val], Y_train_binary[val]),
                        epochs=40, batch_size=50, verbose=0)
    train_loss_history.append(history.history['loss'])
    val_loss_history.append(history.history['val_loss'])

# Plot Model Loss
plt.figure(figsize=(8, 6))
for i in range(len(train_loss_history)):
    plt.plot(train_loss_history[i], label=f"Fold {i+1} (Train)")
    plt.plot(val_loss_history[i], label=f"Fold {i+1} (Validation)")

plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
### END CODE HERE ###

NameError: name 'Sequential' is not defined

## Naive Bayes

In this part, you will use the `GaussianNB` classifier to classify the data. You should not change the default parameters of this classifier. First, train the classifier on the training set and then find the accuracy of it on the test set.

**Question**: What is the accuracy of this method on test set?

**Answer**:

In [7]:
### START CODE HERE ###

### END CODE HERE ###

## LDA (Linear Discriminant Analysis)

In this part, you will use the `LinearDiscriminantAnalysis` classifier to classify the data. You should not change the default parameters of this classifier. First, train the classifier on the training set and then find the accuracy of it on the test set.

**Question**: What is the accuracy of this method on test set?

**Answer**:

In [8]:
### START CODE HERE ###

### END CODE HERE ###

## Conclution

**Question**: What is the best method for classifying this dataset? What is the best accuracy on the test set?

**Answer**: